In [6]:
import os, time, pathlib
from dotenv import load_dotenv
from playwright.sync_api import sync_playwright, expect


In [7]:

load_dotenv()
BASE_URL = "https://tracking.magaya.com/?orgname=37508#livetrack"          # e.g., login page
USERNAME = "lucygboviet"
PASSWORD = "LGarcia27834"
DOWNLOAD_DIR = pathlib.Path("downloads")
DOWNLOAD_DIR.mkdir(exist_ok=True)

In [8]:

# start 
def main():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False)  # set False to see the browser
        context = browser.new_context(accept_downloads=True)
        page = context.new_page()

        # 1) Log in
        page.goto(BASE_URL, wait_until="domcontentloaded")

        # Example: fill login form. Adjust selectors  to your page.
        page.get_by_label("Username").fill(USERNAME)
        page.get_by_label("Password").fill(PASSWORD)
        page.get_by_role("button", name="Sign in").click()
        time.sleep(15)
        # Wait for a known element that confirms login succeeded
        page.wait_for_load_state("networkidle")   # or use a specific locator:
        # expect(page.get_by_role("heading", name="Dashboard")).to_be_visible()

        # 2) Navigate to the download page
        page.goto("https://example.com/reports/daily", wait_until="domcontentloaded")

        # Optional: set filters/date, etc.
        # page.get_by_label("From Date").fill("2025-10-01")
        # page.get_by_label("To Date").fill("2025-10-06")
        # page.get_by_role("button", name="Apply").click()
        # page.wait_for_load_state("networkidle")

        # 3) Trigger the download (Playwright captures the file reliably)
        with page.expect_download() as dl_info:
            # Prefer role/text locators over dynamic IDs
            page.get_by_role("button", name="Download CSV").click()
        download = dl_info.value

        # 4) Save the file with a timestamp
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        suggested = download.suggested_filename or "report.csv"
        safe_name = f"{timestamp}_{suggested}"
        target_path = DOWNLOAD_DIR / safe_name
        download.save_as(target_path)

        print(f"Saved: {target_path.resolve()}")

        context.close()
        browser.close()

if __name__ == "__main__":
    main()


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.